In [132]:
import os
os.chdir('/var/www/html/predicationScript')
# os.getcwd()

In [133]:
import os
import re
import sys
import math
import pandas as pd
import numpy as np
import pickle

rndNum = '931'
uniID = 'P51688'

# rndNum = '672'
# uniID = 'P34059'

def biological_feature(df):
    dom=pd.read_csv('../domain.csv')
    rdv=pd.read_csv('../residual_volume.csv')
    dd_residual_vol=[]
    aa_RV_dict=dict(rdv.values)
    
    # Residual Volume
    for i,j in zip(df["Mutant"],df["Wild"]):
        # Mutant Residue
        if len(i) != 1: 
            mut_vol = math.fsum([float(aa_RV_dict.get(x)) for x in [*i]])
        else: 
            mut_vol = aa_RV_dict.get(i)
            
        # Wild Residue
        if len(j) != 1: 
            wild_vol = math.fsum([float(aa_RV_dict.get(x)) for x in [*j]])
        else: 
            wild_vol = aa_RV_dict.get(j)
        dd_residual_vol.append(mut_vol-wild_vol)
    df["dd_changeinresidualvolume"]=dd_residual_vol
    
    # # Domain and Modification from Residue to Residue
    # dom_dom=dict(dom.drop("Modification",axis=True,inplace=False).values)
    # dom_mod=dict(dom.drop("Domain",axis=True,inplace=False).values)
    # Domain=[]
    # Modification=[]
    # for i in list(df["pos"]):
    #     i=int(i)
    #     Domain.append(dom_dom.get(i))
    #     Modification.append(dom_mod.get(i))
    # df["Domain"]=Domain
    # df["Modification"]=Modification

    # Polarity and Hydrophobicity
    pol=pd.read_csv('../polarity.csv')
    hydrophobicity=pd.read_csv('../hydrophobicity.csv')
    aa_pol_dic=dict(pol.values)
    aa_hydro_dic=dict(hydrophobicity.values)
    dd_pol=[]
    dd_hydrophobicity=[]
    
    # Polarity
    for i,j in zip(df["Mutant"],df["Wild"]):
        # Mutant Residue
        if len(i) != 1: 
            mut_pol = math.fsum([float(aa_pol_dic.get(x)) for x in [*i]])
        else: 
            mut_pol = aa_pol_dic.get(i)
            
        # Wild Residue
        if len(j) != 1: 
            wild_pol = math.fsum([float(aa_pol_dic.get(x)) for x in [*j]])
        else: 
            wild_pol = aa_pol_dic.get(j)
        dd_pol.append(mut_pol-wild_pol)
        
    # Hydrophobicity
    for i,j in zip(df["Mutant"],df["Wild"]):
        # Mutant Residue
        if len(i) != 1: 
            mut_hydro = math.fsum([float(aa_hydro_dic.get(x)) for x in [*i]])
        else: 
            mut_hydro = aa_hydro_dic.get(i)
            
        # Wild Residue
        if len(j) != 1: 
            wild_hydro = math.fsum([float(aa_hydro_dic.get(x)) for x in [*j]])
        else: 
            wild_hydro = aa_hydro_dic.get(j)
        dd_hydrophobicity.append(mut_hydro-wild_hydro)

    df["dd_polarity"]=dd_pol
    df["dd_hydrophobicity"]=dd_hydrophobicity
    return df

def create_class(df):
    mutation=df["Mutation"]
    mutation=list(mutation)
    wild=[]
    atlaa=[]
    pos=[]
    for ele in mutation:
        eleLst = re.split('(\d+)',ele)
        wild.append(eleLst[0])
        atlaa.append(eleLst[2])
        pos.append(eleLst[1])
    return([wild,atlaa,pos])

prntDir = os.getcwd().rsplit('/',1)[0]
os.chdir(f"{prntDir}/fileUpload/{rndNum}/outFiles")
tools = sorted(os.listdir())
feature = []
for i in tools:
    feature.append(i.split('.')[0] + '_score')
df = pd.DataFrame()
Mut = pd.read_csv('../mutation.txt', header=None)
df["Mutation"] = Mut
empty_df = []
for i,j in zip(tools,feature):
    if pd.read_csv(i).empty:
        empty_df = i
    else:
        outF = pd.read_csv(i)
        df[j] =  df['Mutation'].map(outF.set_index('Mutation')['Score'])
        df = df.fillna(0)
mut_info = create_class(df)
df.insert(1,"Wild",mut_info[0])
df.insert(2,"Mutant",mut_info[1])
df.insert(3,"pos",list(map(int, mut_info[2])))
# new_df = biological_feature(df)
new_df = df

# adding Clinical Features to the dataframe
cliniFile = pd.read_csv('../MPS_IIIA_Clinical_Significance.csv', names=['Mut','clinical'])
new_df['clinical_phenotype'] = new_df.Mutation.map(cliniFile.set_index('Mut')['clinical']).fillna(0).astype(int)

# clinPheno = []
# import requests
# url='https://rest.uniprot.org/uniprot/'+uniID+''
# data = requests.get(url).json()
# for i in range(len(data['features'])):
#     if data['features'][i]['type'] == 'Natural variant':
#         if 'severe' in data['features'][i]['description']:
#             clinPheno.append(data['features'][i]['location']['start']['value'])
# commSet = list(set(clinPheno) & set(list(map(int, new_df['pos'].tolist()))))
# new_df['clinical_phenotype'] = ''
# c = 0
# for i in list(new_df['pos']):
#     if i in commSet:
#         new_df['clinical_phenotype'][c] = 1
#     else:
#         new_df['clinical_phenotype'][c] = 0
#     c+=1
# new_df.to_csv("../train_set.csv",index=False)

In [136]:
# new_df

In [137]:
X = new_df.drop(["Mutation","Wild","Mutant","pos","clinical_phenotype"],axis=True).apply(pd.to_numeric, errors = 'coerce')
y = pd.DataFrame(new_df['clinical_phenotype'], columns=['clinical_phenotype']).apply(pd.to_numeric, errors = 'coerce')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 32)
model = LogisticRegression()
ss = StandardScaler()
X_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
matthews_corrcoef(y_test, y_pred)

# Dump model for MPS_III_A
pickle.dump(model, open('../MPS_III_A.pkl', 'wb'))
pickle.dump(ss, open('../Scaler_MPS_III_A.pkl', 'wb'))

# for i in range(1, 80):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = i)
#     model = LogisticRegression()
#     ss = StandardScaler()
#     X_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)
#     X_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     print(f"{i}: {matthews_corrcoef(y_test, y_pred)}")

# model.score(X_test, y_test)
# Dump model for MPS_III_A
# pickle.dump(model, open('../MPS_III_A.pkl', 'wb'))
# pickle.dump(ss, open('../Scaler_MPS_III_A.pkl', 'wb'))

# print(model.predict(X_test))
# print(list(y_test.clinical_phenotype))

In [43]:
X = new_df.drop(["Mutation","Wild","Mutant","pos","clinical_phenotype"],axis=True)
y = pd.DataFrame(new_df['clinical_phenotype'], columns=['clinical_phenotype'])
ConvX = X.apply(pd.to_numeric, errors = 'coerce')
ConvY = y.apply(pd.to_numeric, errors = 'coerce')

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# X_train, X_test, y_train, y_test = train_test_split(ConvX, ConvY, test_size=0.2, random_state=42)
# ss = StandardScaler()
# ss_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)
# ss_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns)

ss = StandardScaler()
ss_train = pd.DataFrame(ss.fit_transform(ConvX),columns=ConvX.columns)
model = LogisticRegression()
model.fit(ss_train, ConvY)

# # Dump model for MPS_IV_A 
# pickle.dump(model, open('../MPS_IV_A.pkl', 'wb'))
# pickle.dump(ss, open('../Scaler_MPS_IV_A.pkl', 'wb'))

# Dump model for MPS_III_A
pickle.dump(model, open('../MPS_III_A.pkl', 'wb'))
pickle.dump(ss, open('../Scaler_MPS_III_A.pkl', 'wb'))

# y_predicted = model.predict(ss_test)
# dumm = np.where(y_predicted == 0,"neutral","severe")
# test_mut["predicted"] = dumm
# prob=model.predict_proba(ss_test)
# LR_prob=[]
# for i,j in zip(y_predicted,prob):
#     LR_prob.append(j[i])
# test_mut["probability"]=LR_prob
# result = pd.DataFrame(map(getMutation, sorted(X_test.index)), columns=['Mutation'])
# final_df = pd.concat([result,test_mut],axis=1)

In [141]:
# final_df

In [875]:
confusion_matrix(y_test, y_predicted)

array([[8, 3],
       [1, 1]])

In [876]:
print(y_test)
print(y_predicted)

    clinical_phenotype
53                   0
60                   1
0                    0
45                   0
5                    1
61                   0
16                   0
12                   0
64                   0
30                   0
33                   0
9                    0
41                   0
[0 0 0 0 1 0 0 0 1 1 1 0 0]


In [874]:
from sklearn.metrics import recall_score, matthews_corrcoef
print(recall_score(y_test, y_predicted))
print(matthews_corrcoef(y_test, y_predicted))

0.5
0.17766726362967536


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def getMutation(indexNum):
    res = new_df['Mutation'][indexNum]
    return res

X = new_df.drop(["Mutation","Wild","Mutant","pos","clinical_phenotype"],axis=True)
y = pd.DataFrame(new_df['clinical_phenotype'], columns=['clinical_phenotype'])
ConvX = X.apply(pd.to_numeric, errors = 'coerce')
ConvY = y.apply(pd.to_numeric, errors = 'coerce')
X_train, X_test, y_train, y_test = train_test_split(
    ConvX, ConvY, test_size=0.2, random_state=42)

ss = StandardScaler()
ss_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)
ss_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns)
model = LogisticRegression(penalty='l2',dual=False,tol=1e-8,
                      C=0.05,fit_intercept=True,class_weight={0:0.957,1:2.6},random_state=14,solver='liblinear')
test_mut = pd.DataFrame()
model.fit(ss_train,y_train)
y_predicted=model.predict(ss_test)
dumm=np.where(y_predicted==0,"neutral","severe")
test_mut["predicted"]=dumm
prob=model.predict_proba(ss_test)
LR_prob=[]
for i,j in zip(y_predicted,prob):
    LR_prob.append(j[i])
test_mut["probability"]=LR_prob
result = pd.DataFrame(map(getMutation, sorted(X_test.index)), columns=['Mutation'])
final_df=pd.concat([result,test_mut],axis=1)
print(final_df)
final_df.to_csv("../ModelPrediction.csv",index=False)

     Mutation predicted  probability
0         M1A   neutral     0.969416
1         M1D   neutral     0.946433
2         M1H   neutral     0.927172
3         M1L   neutral     0.906791
4         M1F   neutral     0.981369
...       ...       ...          ...
1903    E501D   neutral     0.909142
1904    E501K   neutral     0.995764
1905    L502H   neutral     0.992237
1906    L502F   neutral     0.994555
1907    L502Y   neutral     0.945940

[1908 rows x 3 columns]


/home/software/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
os.getcwd()

'/var/www/html/fileUpload/672/outFiles'

In [16]:
# y_test

In [815]:
# model
# X_test

In [37]:
import pickle
pickle.dump(model, open('../MPS_III_A.pkl', 'wb'))

In [816]:
essColumn = ['duet_score', 'foldX_score', 'iMutant_score', 'mCSM_score', 'maestro_score', 
             'metaSNP_score', 'muPro_score', 'polyphen2_score', 'sdm_score', 'snpsGO_score', 
             'dd_changeinresidualvolume', 'Domain', 'Modification', 'dd_polarity',
             'dd_hydrophobicity']
X_test_2 = X_test.drop(["duet_score","iMutant_score"],axis=True)

In [818]:
# X_test.head()

In [820]:
for ec in enumerate(essColumn, 0):
    if ec[1] not in X_test_2.columns:
        X_test_2.insert(ec[0], ec[1], 0)

In [821]:
len(X_test_2.columns)

15

In [822]:
X_test_2.head()

,duet_score,foldX_score,iMutant_score,mCSM_score,maestro_score,metaSNP_score,muPro_score,polyphen2_score,sdm_score,snpsGO_score,dd_changeinresidualvolume,Domain,Modification,dd_polarity,dd_hydrophobicity
53,0,3.30569,0,-0.824,-0.07693,0.174,-0.240278,0.001,-0.38,0.068,19.4,2,0,0.0,0.60
60,0,2.84244,0,-0.586,0.18647,0.853,-0.622768,1.000,-0.03,0.879,26.3,3,0,-5.1,2.05
0,0,2.04868,0,-0.423,0.17416,0.808,-0.420749,1.000,-0.29,0.781,16.3,1,0,-0.7,0.01
45,0,3.09165,0,0.143,1.06610,0.440,-1.086024,1.000,-0.86,0.118,20.4,2,0,-1.0,0.68
5,0,17.97060,0,-1.241,-0.15722,0.814,0.066215,0.997,-0.11,0.471,80.5,1,0,-3.8,2.60


In [435]:
# 

In [823]:
# X_test_2 = X_test_2.iloc[:, :-5]

In [436]:
# X_test_2.head()

In [836]:
pickled_model = pickle.load(open('MPS_III_A.pkl', 'rb'))
# pickled_model.predict(X_test_2)
list(pickled_model.predict_proba(X_test_2))

[array([0.60177062, 0.39822938]),
 array([0.49157132, 0.50842868]),
 array([0.58998951, 0.41001049]),
 array([0.66318006, 0.33681994]),
 array([0.58752686, 0.41247314]),
 array([0.67119061, 0.32880939]),
 array([0.63190694, 0.36809306]),
 array([0.80529629, 0.19470371]),
 array([0.64664707, 0.35335293]),
 array([0.56612999, 0.43387001]),
 array([0.4917133, 0.5082867]),
 array([0.55558095, 0.44441905]),
 array([0.46134536, 0.53865464])]

In [877]:
# pickled_model

In [322]:
# ####### Feature Scaling ######
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# classifier = LogisticRegression()
# parameter = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[0.5,1,1.5,2,4,5,6,7,10,20,30,50], 'max_iter': [100, 200, 300]}
# # {'C': 2, 'max_iter': 100, 'penalty': 'l2'} #### Got this parameter with 0.72 accuracy rate
# classifier_regressor = GridSearchCV(classifier, param_grid = parameter, scoring='accuracy', cv=5)
# classifier_regressor.fit(ConvX, ConvY)
# print(classifier_regressor.best_params_)
# print(classifier_regressor.best_score_)
# y_pred = classifier_regressor.predict(ss_test)
# accr_score = accuracy_score(y_pred, y_test)
# print(accr_score)

In [323]:
# from sklearn.metrics import confusion_matrix
# tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()
# tn, fp, fn, tp

In [324]:
# acc  = tp + tn / tp + fp + tn + fn
# acc

In [506]:
# df.head()
# df.to_csv("../test_purpose.csv",index=False)

In [561]:
dom=pd.read_csv('../domain.csv')
rdv=pd.read_csv('../residual_volume.csv')
dd_residual_vol=[]
aa_RV_dict=dict(rdv.values)
# for i,j in zip(df["Mutant"],df["Wild"]):
#     mut_vol=aa_RV_dict.get(i)
#     wild_vol=aa_RV_dict.get(j)
    # print(wild_vol)

In [584]:
# aa_RV_dict
# test_mut_vol = aa_RV_dict.get('N')
# # print([*strMut])
# test_mut_vol

In [554]:
# strMut = df["Mutant"][54]
# if len(strMut) > 0:
#     print(strMut.split())
# else:
#     print('NO')
# # len(strMut)

In [577]:
# if len(strMut) != 1: 
#     sptStr = [*strMut]
# else: 
#     print("No")

In [760]:
# sptStr
# if len(strMut) != 1: 
#     print('yes') 
# else:
#     print('No')
# print(abc)
resVol

[91.9, 89.7, 109.1]

In [762]:
import math
resVol = math.fsum([float(aa_RV_dict.get(x)) for x in [*strMut]])
print(resVol)

290.7


In [ ]:
290.70000000000005

In [ ]:
def biological_feature(df):
    dom=pd.read_csv('../domain.csv')
    rdv=pd.read_csv('../residual_volume.csv')
    dd_residual_vol=[]
    aa_RV_dict=dict(rdv.values)
    for i,j in zip(df["Mutant"],df["Wild"]):
        mut_vol=aa_RV_dict.get(i)
        wild_vol=aa_RV_dict.get(j)
        dd_residual_vol.append(mut_vol-wild_vol)
    df["dd_changeinresidualvolume"]=dd_residual_vol
    dom_dom=dict(dom.drop("Modification",axis=True,inplace=False).values)
    dom_mod=dict(dom.drop("Domain",axis=True,inplace=False).values)
    Domain=[]
    Modification=[]
    for i in list(df["pos"]):
        i=int(i)
        Domain.append(dom_dom.get(i))
        Modification.append(dom_mod.get(i))
    df["Domain"]=Domain
    df["Modification"]=Modification

    ####polarity######
    pol=pd.read_csv('../polarity.csv')
    hydrophobicity=pd.read_csv('../hydrophobicity.csv')
    aa_pol_dic=dict(pol.values)
    aa_hydro_dic=dict(hydrophobicity.values)
    dd_pol=[]
    dd_hydrophobicity=[]
    for i,j in zip(df["Mutant"],df["Wild"]):
        mut_pol=aa_pol_dic.get(i)
        wild_pol=aa_pol_dic.get(j)
        dd_pol.append(mut_pol-wild_pol)
    for i,j in zip(df["Mutant"],df["Wild"]):
        mut_hydro=aa_hydro_dic.get(i)
        wild_hydro=aa_hydro_dic.get(j)
        dd_hydrophobicity.append(mut_hydro-wild_hydro)

    df["dd_polarity"]=dd_pol
    df["dd_hydrophobicity"]=dd_hydrophobicity
    return df

In [37]:
import os
import warnings
os.chdir('/var/www/html')
# os.getcwd()
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

rndNum = 480
selMod = "SGSH"
# rndNum = sys.argv[1]
# selMod = sys.argv[2]
# essColumn = ['duet_score', 'foldX_score', 'iMutant_score', 'mCSM_score', 'maestro_score', 
#              'metaSNP_score', 'muPro_score', 'polyphen2_score', 'sdm_score', 'snpsGO_score', 
#              'dd_changeinresidualvolume', 'Domain', 'Modification', 'dd_polarity',
#              'dd_hydrophobicity']
prntDir = os.getcwd()
os.chdir(f"{prntDir}/fileUpload/{rndNum}")
if (selMod == 'SGSH'):
    model = 'MPS_III_A'
    StandardScalar = 'Scaler_MPS_III_A'
else:
    model = 'MPS_IV_A'
    StandardScalar = 'Scaler_MPS_IV_A'

# pre-Processing data for the prediction
df=pd.read_csv("train_set.csv")
X = df.drop(["Mutation","Wild","Mutant","pos","clinical_phenotype"],axis=True)
y = pd.DataFrame(df['clinical_phenotype'], columns=['clinical_phenotype'])
ConvX = X.apply(pd.to_numeric, errors = 'coerce')
ConvY = y.apply(pd.to_numeric, errors = 'coerce')
# ss = pickle.load(open(f"{prntDir}/featureModel/{StandardScalar}.pkl", 'rb'))
# ConvX = pd.DataFrame(ss.transform(ConvX),columns=ConvX.columns)

# for ec in enumerate(essColumn, 0):
#     if ec[1] not in ss_test.columns:
#         ss_test.insert(ec[0], ec[1], 0)

# maestro, mCSM, muPro
ConvX = ConvX.drop(['mCSM_score', 'maestro_score', 'muPro_score'], axis=1)

# Predication 
new_df = pd.DataFrame(df['Mutation'], columns=['Mutation'])
pickled_model = pickle.load(open(f"{prntDir}/featureModel/{model}.pkl", 'rb'))
predLst = pickled_model.predict(ConvX)
dumm=np.where(predLst==0,"Neutral","Severe")
new_df["predicted"]=dumm
prdProbLst = pickled_model.predict_proba(ConvX)
LR_prob = []
for i,j in zip(predLst,prdProbLst):
    LR_prob.append(j[i])
new_df["probability"] = LR_prob
new_df.to_csv("ModelPrediction.csv",index=False)
print(new_df)

  Mutation predicted  probability
0    D179N   Neutral     0.556853
